<a href="https://colab.research.google.com/github/shramana263/ResoluteAI-Python/blob/main/PDF_text_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2 tabula-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 75.0 MB/s eta 0:00:00


In [ ]:
import PyPDF2
import tabula
import json
import os

def extract_pdf_content(pdf_path):
    """
    Extract text and tables from a PDF file and organize into a JSON structure.

    Args:
        pdf_path (str): Path to the PDF file.

    Returns:
        dict: JSON structure with headers and list_items.
    """
    output = {
        "headers": {},
        "list_items": []
    }

    try:
        with open(pdf_path, 'rb') as file:
            #Creating a PDF reader object
            pdf_reader = PyPDF2.PdfReader(file)

            #Extracting text
            text_content = ""
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text_content += page.extract_text() or ""


            text_content = " ".join(text_content.split())
            output["headers"]["extracted_text"] = text_content

            #Extracting tables
            tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True)

            #Processing each table
            for table_index, table in enumerate(tables):
                #Convert table to a list of dictionaries for JSON compatibility
                table_data = table.to_dict(orient='records')
                output["list_items"].append({
                    f"table_{table_index + 1}": table_data
                })

        return output

    except Exception as e:
        print(f"Error processing PDF: {str(e)}")
        return None

def save_to_json(data, output_path):

    try:
        with open(output_path, 'w', encoding='utf-8') as json_file:
            json.dump(data, json_file, indent=4)
        print(f"JSON file saved to {output_path}")
    except Exception as e:
        print(f"Error saving JSON: {str(e)}")

def main():

    from google.colab import files
    print("Please upload your PDF file:")
    uploaded = files.upload()

    #Getting the file path
    pdf_path = list(uploaded.keys())[0]

    #initializing the output file
    output_json_path = "extracted_data.json"
    extracted_data = extract_pdf_content(pdf_path)

    if extracted_data:
        save_to_json(extracted_data, output_json_path)

        # Display
        print("\nExtracted Data:")
        print(json.dumps(extracted_data, indent=4))

        # Download
        files.download(output_json_path)
    else:
        print("Failed to extract data from the PDF.")

if __name__ == "__main__":
    main()

Please upload your PDF file:


Saving Assessment_sample_pdf.pdf to Assessment_sample_pdf.pdf
JSON file saved to extracted_data.json

Extracted Data:
{
    "headers": {
        "extracted_text": "Title: Sample Document This is a sample paragraph describing the document. Table1: Employee Name Age Departme nt Shramana Show 20 Frontend Dev Debshruti Saha 21 Frontend Dev Piku Mondal 21 Backend Dev"
    },
    "list_items": [
        {
            "table_1": [
                {
                    "Name": "Shramana Show",
                    "Age": 20,
                    "Department": "Frontend Dev"
                },
                {
                    "Name": "Debshruti Saha",
                    "Age": 21,
                    "Department": "Frontend Dev"
                },
                {
                    "Name": "Piku Mondal",
                    "Age": 21,
                    "Department": "Backend Dev"
                }
            ]
        }
    ]
}


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
!pip install gdown PyPDF2 tabula-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 46.1 MB/s eta 0:00:00


In [ ]:
import gdown
import PyPDF2
import tabula
import json
import os
from google.colab import files

def extract_pdf_content(pdf_path):
    """
    Extract text, metadata, and tables from a PDF file and organize into a JSON structure.

    Args:
        pdf_path (str): Path to the PDF file.

    Returns:
        dict: JSON structure with headers and list_items.
    """
    output = {
        "headers": {},
        "list_items": []
    }

    try:
        # Open the PDF file
        with open(pdf_path, 'rb') as file:
            # Create a PDF reader object
            pdf_reader = PyPDF2.PdfReader(file)

            # Extract metadata
            metadata = pdf_reader.metadata
            if metadata:
                for key in metadata:
                    clean_key = key.lstrip('/')
                    output["headers"][clean_key] = metadata[key]

            # Extract text from all pages
            text_content = ""
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text_content += page.extract_text() or ""

            # Clean up text
            text_content = " ".join(text_content.split())
            output["headers"]["extracted_text"] = text_content

            # Extract tables using tabula-py
            tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True)

            # Process each table
            for table_index, table in enumerate(tables):
                # Convert table to a list of dictionaries for JSON compatibility
                table_data = table.to_dict(orient='records')
                output["list_items"].append({
                    f"table_{table_index + 1}": table_data
                })

        return output

    except Exception as e:
        print(f"Error processing PDF: {str(e)}")
        return None

def save_to_json(data, output_path):
    """
    Save the extracted data to a JSON file.

    Args:
        data (dict): Data to save.
        output_path (str): Path to save the JSON file.
    """
    try:
        with open(output_path, 'w', encoding='utf-8') as json_file:
            json.dump(data, json_file, indent=4)
        print(f"JSON file saved to {output_path}")
    except Exception as e:
        print(f"Error saving JSON: {str(e)}")

def main():
    """
    Main function to handle user input, download PDF, extract data, and save JSON.
    """
    print("Please enter the Google Drive link to the PDF:")
    drive_link = input()

    # Download the PDF
    output_path = "downloaded_pdf.pdf"
    gdown.download(drive_link, output_path, quiet=False)

    if os.path.exists(output_path):
        extracted_data = extract_pdf_content(output_path)
        if extracted_data:
            save_to_json(extracted_data, "extracted_data.json")
            print("\nExtracted Data:")
            print(json.dumps(extracted_data, indent=4))
            files.download("extracted_data.json")
        else:
            print("Failed to extract data from the PDF.")
    else:
        print("Failed to download the PDF.")

if __name__ == "__main__":
    main()